<a href="https://colab.research.google.com/github/preetamjumech/CNN_from_Scratch/blob/main/Preetam_Saha_05_09_2022_MNIST_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt #.pt = in form of tensors
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/test.pt

--2022-09-05 15:07:12--  https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt [following]
--2022-09-05 15:07:13--  https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47520431 (45M) [application/octet-stream]
Saving to: ‘training.pt’

training.pt         100%[=================

In [2]:
import torch, torchsummary
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam, RMSprop
import numpy as np

In [3]:
(x_train, y_train), (x_test, y_test) = (torch.load('training.pt'),torch.load('test.pt'))

In [4]:
#in pytorch channel of the image second dimension , 1st=batchsize, 2nd = channel, 3rd = width, 4th = height 
#tensorflow channel is the fourth dimension

x_train = x_train.reshape(-1,1,28,28) #1 channel for grey scale
x_test = x_test.reshape(-1,1,28,28) #second 1 for channel , cant build cnn without this information, -1 for batchsize

In [5]:
from PIL import Image

In [6]:
idx=3
Image.fromarray(x_train[idx,0,:,:].numpy())

In [7]:
idx=300
Image.fromarray(x_train[idx,0,:,:].numpy())

In [8]:
class MNIST_CNN(nn.Module):
  def __init__(self):
    super(MNIST_CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=16, kernel_size=3, padding=1 ) #in_channel = channel, out_channels=no of filters, kernel_Size = filter size
    self.conv2 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, padding=1 )
    self.conv3 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3, padding=1 )
    self.conv4 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3, padding=1 )
    self.bn1 = nn.BatchNorm2d(num_features=16)
    self.bn2 = nn.BatchNorm2d(num_features=64)
    self.pool = nn.MaxPool2d(kernel_size=2, stride = 2) #by pooling only reduction happenes, stride = 2 means jump of 2 pixels
    self.h1 = nn.Linear(in_features=1*1*128, out_features= 512)
    self.h2 = nn.Linear(in_features=512, out_features= 256)
    self.h3 = nn.Linear(in_features= 256, out_features= 32)
    self.out = nn.Linear(in_features = 32, out_features=10)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)

    x = self.bn1(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)

    x = F.relu(self.conv3(x))
    x = self.pool(x)

    x = self.bn2(x)
    x = F.relu(self.conv4(x))
    x = self.pool(x)
    #print(x.size())

    x = x.view(x.size()[0],-1)
    #print(x.size())

    x = F.relu(self.h1(x))
    x = F.relu(self.h2(x))
    x = F.relu(self.h3(x))
    x = self.out(x)

    return x



In [9]:
cnn_model = MNIST_CNN()
list(cnn_model.parameters())

[Parameter containing:
 tensor([[[[-0.2961, -0.2786, -0.2676],
           [-0.1988,  0.1258,  0.2130],
           [ 0.3286,  0.0270, -0.0315]]],
 
 
         [[[-0.1522,  0.0857,  0.2052],
           [-0.0135,  0.1384,  0.2465],
           [-0.2061,  0.2273, -0.3003]]],
 
 
         [[[ 0.1291, -0.1142, -0.0600],
           [-0.0831, -0.1270,  0.2767],
           [-0.0110,  0.1529,  0.0528]]],
 
 
         [[[-0.2403,  0.3324, -0.1415],
           [-0.1795,  0.2097,  0.3105],
           [ 0.2698,  0.1806,  0.1436]]],
 
 
         [[[ 0.3041,  0.2870, -0.0990],
           [-0.0819, -0.3175, -0.1505],
           [-0.2464,  0.2564,  0.1072]]],
 
 
         [[[-0.2280,  0.1939,  0.1695],
           [-0.1088,  0.3291, -0.2146],
           [-0.1906,  0.2814, -0.3258]]],
 
 
         [[[-0.2537,  0.0983,  0.0383],
           [-0.0856,  0.0697,  0.0867],
           [ 0.0890,  0.1012, -0.1497]]],
 
 
         [[[-0.1095, -0.2853, -0.0629],
           [-0.3049, -0.1571,  0.1195],
           [-0.

In [10]:
torchsummary.summary(cnn_model,input_size=(1,28,28),device = "cpu",batch_size=1) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 16, 28, 28]             160
         MaxPool2d-2            [1, 16, 14, 14]               0
       BatchNorm2d-3            [1, 16, 14, 14]              32
            Conv2d-4            [1, 32, 14, 14]           4,640
         MaxPool2d-5              [1, 32, 7, 7]               0
            Conv2d-6              [1, 64, 7, 7]          18,496
         MaxPool2d-7              [1, 64, 3, 3]               0
       BatchNorm2d-8              [1, 64, 3, 3]             128
            Conv2d-9             [1, 128, 3, 3]          73,856
        MaxPool2d-10             [1, 128, 1, 1]               0
           Linear-11                   [1, 512]          66,048
           Linear-12                   [1, 256]         131,328
           Linear-13                    [1, 32]           8,224
           Linear-14                   

In [11]:
opt = Adam(params=cnn_model.parameters())

In [12]:
train_loader = DataLoader(TensorDataset(x_train.float(),y_train),batch_size=64,shuffle=True)

In [13]:
n_epochs = 50
cnn_model = cnn_model.to("cuda")
for epoch in range(n_epochs):
  training_loss = 0.0
  for batch, target in train_loader:
    batch = batch.to("cuda")
    target = target.to("cuda")

    opt.zero_grad()
    output = cnn_model(batch)
    #probs = torch.softmax(output,dim=1)
    loss = F.cross_entropy(output, target)
    loss.backward()

    opt.step()
    training_loss += loss.item()

  if (epoch+1)%5 ==0:
    print(f"Training loss after {epoch+1} is : {training_loss}")


Training loss after 5 is : 25.873621437465772
Training loss after 10 is : 13.050845159384153
Training loss after 15 is : 10.858933879230449
Training loss after 20 is : 7.971546850550247
Training loss after 25 is : 7.610025110563079
Training loss after 30 is : 5.760909594083098
Training loss after 35 is : 5.40191063415261
Training loss after 40 is : 3.5901967466724862
Training loss after 45 is : 4.414646754060712
Training loss after 50 is : 2.9493699161900757


In [14]:
preds_test = cnn_model(x_test.float().to("cuda")).detach().cpu().numpy()

In [15]:
preds = np.argmax(preds_test,axis = 1)

In [16]:
import pandas as pd

In [17]:
pd.crosstab(preds, y_test.numpy())

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,978,0,0,0,0,1,3,0,1,0
1,0,1133,0,0,0,0,1,2,0,2
2,2,0,1028,0,1,0,1,2,0,0
3,0,0,0,1003,0,5,0,0,0,0
4,0,0,0,0,969,0,0,0,0,4
5,0,0,0,5,0,883,1,0,1,1
6,0,0,0,0,3,1,950,0,0,0
7,0,2,3,2,2,0,0,1022,0,0
8,0,0,1,0,1,0,2,0,971,3
